In [1]:
import pandas as pd
import numpy as np
import pickle

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import shap
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn import metrics
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool

In [8]:
with open('./Data/data.pickle', 'rb') as f:
    df = pickle.load(f)
    f.close()

In [9]:
df_raw = pd.read_csv('./Data/bank-additional-full.csv', encoding='utf8', sep=';')

In [10]:
drop_idx = df_raw[(df_raw['job'] == 'unknown') | (df_raw['marital'] == 'unknown') | (df_raw['education'] == 'unknown') | (df_raw['housing'] == 'unknown') | (df_raw['loan'] == 'unknown')].index
df_raw.drop(df_raw.index[drop_idx], inplace=True)

In [11]:
X_raw = df_raw.drop(columns=['y'])
y_raw = df_raw['y']

In [12]:
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X_raw, y_raw, test_size=0.2, random_state=42)
X_train_raw, X_valid_raw, y_train_raw, y_valid_raw = train_test_split(X_train_raw, y_train_raw, test_size=0.25, random_state=42)

In [13]:
X_train_raw

age            job   marital            education  default housing  \
21958   36     technician   married    university.degree       no      no   
1941    24    blue-collar   married             basic.9y       no     yes   
20888   47      housemaid   married          high.school  unknown     yes   
2041    49     management  divorced          high.school  unknown      no   
6924    30  self-employed    single    university.degree       no     yes   
...    ...            ...       ...                  ...      ...     ...   
40537   27         admin.    single          high.school       no      no   
29216   37     technician  divorced          high.school       no      no   
3859    32    blue-collar   married             basic.4y  unknown     yes   
37858   28        student    single  professional.course       no      no   
18522   44     technician   married  professional.course       no     yes   

      loan    contact month day_of_week  duration  campaign  pdays  previous  \
21958  yes   cellular   aug         wed        62         2    999         0   
1941   yes  telephone   may         fri       101         6    999         0   
20888   no   cellular   aug         thu       107         1    999         0   
2041    no  telephone   may         mon       124         1    999         0   
6924    no  telephone   may         thu       244         2    999         0   
...    ...        ...   ...         ...       ...       ...    ...       ...   
40537   no  telephone   aug         fri      3785         1    999         0   
29216   no   cellular   apr         fri       188         1    999         0   
3859    no  telephone   may         fri       469         2    999         0   
37858   no   cellular   aug         fri        36         2    999         0   
18522   no  telephone   jul         thu       168         1    999         0   

          poutcome  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  \
21958  nonexistent           1.4          93.444          -36.1      4.964   
1941   nonexistent           1.1          93.994          -36.4      4.855   
20888  nonexistent           1.4          93.444          -36.1      4.964   
2041   nonexistent           1.1          93.994          -36.4      4.857   
6924   nonexistent           1.1          93.994          -36.4      4.860   
...            ...           ...             ...            ...        ...   
40537  nonexistent          -1.7          94.027          -38.3      0.888   
29216  nonexistent          -1.8          93.075          -47.1      1.405   
3859   nonexistent           1.1          93.994          -36.4      4.859   
37858  nonexistent          -2.9          92.201          -31.4      0.825   
18522  nonexistent           1.4          93.918          -42.7      4.968   

       nr.employed  
21958       5228.1  
1941        5191.0  
20888       5228.1  
2041        5191.0  
6924        5191.0  
...            ...  
40537       4991.6  
29216       5099.1  
3859        5191.0  
37858       5076.2  
18522       5228.1  

[22947 rows x 20 columns]

In [14]:
X = df.drop(columns=['y'])
y = df['y']

In [15]:
X['job_rate'] = X_raw['job']

In [16]:
X

age_range_20  age_range_30  age_range_40  age_range_50  age_range_60  \
0                 0             0             0             1             0   
1                 0             0             0             1             0   
2                 0             1             0             0             0   
3                 0             1             0             0             0   
4                 0             0             0             1             0   
...             ...           ...           ...           ...           ...   
41183             0             0             0             0             1   
41184             0             0             1             0             0   
41185             0             0             0             1             0   
41186             0             0             1             0             0   
41187             0             0             0             0             1   

          job_rate  marital_divorced  marital_married  marital_single  \
0        housemaid                 0                1               0   
1         services                 0                1               0   
2         services                 0                1               0   
3           admin.                 0                1               0   
4         services                 0                1               0   
...            ...               ...              ...             ...   
41183      retired                 0                1               0   
41184  blue-collar                 0                1               0   
41185      retired                 0                1               0   
41186   technician                 0                1               0   
41187      retired                 0                1               0   

       education_basic.4y  ...  pdays_1  previous_0  previous_1  \
0                       1  ...        1           1           0   
1                       0  ...        1           1           0   
2                       0  ...        1           1           0   
3                       0  ...        1           1           0   
4                       0  ...        1           1           0   
...                   ...  ...      ...         ...         ...   
41183                   0  ...        1           1           0   
41184                   0  ...        1           1           0   
41185                   0  ...        1           1           0   
41186                   0  ...        1           1           0   
41187                   0  ...        1           0           1   

       poutcome_non-success  poutcome_success  emp.var.rate  cons.price.idx  \
0                         1                 0           1.1          93.994   
1                         1                 0           1.1          93.994   
2                         1                 0           1.1          93.994   
3                         1                 0           1.1          93.994   
4                         1                 0           1.1          93.994   
...                     ...               ...           ...             ...   
41183                     1                 0          -1.1          94.767   
41184                     1                 0          -1.1          94.767   
41185                     1                 0          -1.1          94.767   
41186                     1                 0          -1.1          94.767   
41187                     1                 0          -1.1          94.767   

       cons.conf.idx  euribor3m  nr.employed  
0              -36.4      4.857       5191.0  
1              -36.4      4.857       5191.0  
2              -36.4      4.857       5191.0  
3              -36.4      4.857       5191.0  
4              -36.4      4.857       5191.0  
...              ...        ...          ...  
41183          -50.8      1.028       4963.6  
41184          -50.8      1.028       4963.6  
4118

In [45]:
y = y.astype('int64')

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [48]:
temp = pd.concat([X_train, y_train], axis=1)
job_mean = temp.groupby('job_rate')['y'].mean()

In [52]:
X_train['job_rate'] = X_train['job_rate'].map(job_mean)
X_test['job_rate'] = X_test['job_rate'].map(job_mean)
X_valid['job_rate'] = X_valid['job_rate'].map(job_mean)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [77]:
flist = [(X_train, 'X_train'), (X_valid, 'X_valid'), (X_test, 'X_test'), (y_train, 'y_train'), (y_valid, 'y_valid'), (y_test, 'y_test')]

for obj in flist:
    with open(f'./Data/{obj[1]}.pkl', 'wb') as f:
        pickle.dump(obj[0], f)
        f.close()